In [1]:
import battery_parser as bp
import os
import pandas as pd
import numpy as np
import re
import copy
import itertools

In [2]:
#get all files from directory
root_dir  = r'D:\!Science\Физтех\Циклирования\28.05.24 - результаты'
files = bp.list_files(directory=root_dir, filetype='xlsx')

#parse info
patterns = [r'([A-Z]{3})', r'([^\\]*)', r'([0-9][^\\]*)', r'.?([0-9]?[0-9])', r'.*-(13[0-9])-.-([0-9]).*-([0-9]{14})']
column_names = ['Chem','Tech', 'Step', 'Pouch','Potentiostat', 'Channel', 'Datetime_id']
pattern = r'\\'.join(patterns)
parser = bp.importing.Regex_parse()
result = parser(strings=files,pattern=pattern,column_names= column_names)

#data enhancement
type_convert = {i:str for i in result.columns}
result = result.astype({'Pouch':int}, copy=True)
result = result.astype(type_convert, copy=True)

#Create ID - unique for any pouch
result['ID'] = result['Chem'] + result['Pouch']
result.insert(0, 'ID', result.pop('ID'))

#Add info about thermos and cycling protocol
thermos = {'1':['NMC14', 'NMC15', 'LTO7', 'LTO6', 'LFP4', 'LFP5'],
           '2':['LFP2', 'LFP3', 'NMC6', 'NMC7', 'LTO11', 'LTO12'],
           '3':['NMC12', 'NMC13', 'LTO13', 'LTO3', 'LFP6', 'LFP7']}
protocol = {'gost45':['NMC6', 'NMC7', 'LTO11', 'LTO12', 'LFP2', 'LFP3'],
            'gost60':['NMC12', 'NMC13', 'LTO13', 'LTO3', 'LFP6', 'LFP7'],
            'Imax45':['NMC14', 'NMC15', 'LTO7', 'LTO6', 'LFP4', 'LFP5'],
            'CCroom':['NMC10', 'NMC11', 'LTO4', 'LTO5', 'LFP1', 'LFP8']}

if bp.modifications.check_dict_intersection(thermos) or bp.modifications.check_dict_intersection(protocol):
    print('Dict Intersection!')

thermos_creator = bp.modifications.Find_key(thermos)
technique_creator = bp.modifications.Find_key(protocol)

result['Protocol'] = result['ID'].apply(technique_creator)
result.insert(2, 'Protocol', result.pop('Protocol'))
result['Thermos'] = result['ID'].apply(thermos_creator)
result.insert(6, 'Thermos', result.pop('Thermos'))

#Create Step_id - unique value for all steps
result.sort_values('Step', inplace=True)

step_id = [bp.modifications.step_id_creator(pouch) for name, pouch in result.groupby('ID')]
result['Step_id'] = pd.concat(step_id, )
result.insert(4, 'Step_id', result.pop('Step_id'))


result

Warning! For 
 D:\!Science\Физтех\Циклирования\28.05.24 - результаты\LFP\Заявка №73 Старение+45 2-3\4. п.5 Циклирование 28 дней\разряд Приложение Б\BTS82-133-1-7-99-20240416085854-Разряд профилем Б.xlsx 
 found 0 entries!
Warning! For 
 D:\!Science\Физтех\Циклирования\28.05.24 - результаты\LFP\Заявка №73 Старение+45 2-3\4. п.5 Циклирование 28 дней\разряд Приложение Б\BTS82-133-1-8-99-20240416085912-Разряд профилем Б.xlsx 
 found 0 entries!


,ID,Chem,Protocol,Tech,Step_id,Step,Pouch,Thermos,Potentiostat,Channel,Datetime_id,Path
60,LTO3,LTO,gost60,7. Заявка № 71 К3 К13,1,1 п1-4.1,3,3,131,7,20240403111709,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
65,LTO13,LTO,gost60,7. Заявка № 71 К3 К13,1,1 п1-4.1,13,3,131,8,20240403111718,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
95,NMC7,NMC,gost45,Заявка №57 KAMAZ 006-007,1,1. Пункт 1-4.1,7,2,133,2,20240314161946,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
91,NMC6,NMC,gost45,Заявка №57 KAMAZ 006-007,1,1. Пункт 1-4.1,6,2,133,1,20240314161930,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
52,NMC15,NMC,Imax45,Заявка №64 C-NMC №014 №015,1,1. Пункт 1-4.1,15,1,131,6,20240325102602,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
...,...,...,...,...,...,...,...,...,...,...,...,...
115,LFP2,LFP,gost45,Заявка №73 Старение+45 2-3,4,5. Испытание рабочих характеристик,2,2,133,7,20240513133440,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
85,LFP6,LFP,gost60,Заявка №75 Старение +60,4,5. Испытание рабочих характеристик,6,3,132,7,20240520102038,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
110,LFP5,LFP,Imax45,Заявка №74 Старение +45 4-5,4,5. Испытание рабочих характеристик,5,1,133,6,20240513144931,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
81,LFP1,LFP,CCroom,Заявка №76 Старение НКУ,4,5. Пункт 6,1,[],132,6,20240514100556,D:\!Science\Физтех\Циклирования\28.05.24 - рез...


In [5]:
#Тест по размеру каждого шага
sizes ={}
for name, dataframe in result.groupby(['ID','Step_id']):
    sizeof = np.sum([os.path.getsize(i) for i in dataframe['Path']])
    sizes[name] = sizeof
pd.Series(sizes)

LFP1  1     27088915
      2      1883962
      3     24417722
      4     40183898
LFP2  1     27169404
             ...    
NMC6  4     14156709
NMC7  1     14390386
      2      1090413
      3    159162047
      4     14074754
Length: 96, dtype: int32

In [3]:
result.sort_values(['Datetime_id', 'Path'], inplace=True)
result

,ID,Chem,Protocol,Tech,Step_id,Step,Pouch,Thermos,Potentiostat,Channel,Datetime_id,Path
91,NMC6,NMC,gost45,Заявка №57 KAMAZ 006-007,1,1. Пункт 1-4.1,6,2,133,1,20240314161930,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
95,NMC7,NMC,gost45,Заявка №57 KAMAZ 006-007,1,1. Пункт 1-4.1,7,2,133,2,20240314161946,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
22,NMC10,NMC,CCroom,"Заявка № 60 Образцы C-NMC № 010, 011",1,1. Пункт 1-4.1,10,[],131,1,20240325100127,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
28,NMC11,NMC,CCroom,"Заявка № 60 Образцы C-NMC № 010, 011",1,1. Пункт 1-4.1,11,[],131,2,20240325100138,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
34,NMC12,NMC,gost60,Заявка№ 61,1,1. п 1- п4.1,12,3,131,3,20240325101703,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
...,...,...,...,...,...,...,...,...,...,...,...,...
59,NMC13,NMC,gost60,Заявка№ 61,4,4. Испытание рабочих характеристик,13,3,131,6,20240520100859,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
64,LTO3,LTO,gost60,7. Заявка № 71 К3 К13,4,4. Испытание рабочих характеристик,3,3,131,7,20240520101600,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
77,LTO13,LTO,gost60,7. Заявка № 71 К3 К13,4,4. Испытание рабочих характеристик,13,3,132,4,20240520101645,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
85,LFP6,LFP,gost60,Заявка №75 Старение +60,4,5. Испытание рабочих характеристик,6,3,132,7,20240520102038,D:\!Science\Физтех\Циклирования\28.05.24 - рез...


In [4]:
result_path = r'D:\!Science\Физтех\Циклирования\Результаты\File_statistics.csv'
bp.exporting.save_experiment(result, result_path)
result = bp.exporting.load_experiment(result_path)
result

,ID,Chem,Protocol,Tech,Step_id,Step,Pouch,Thermos,Potentiostat,Channel,Datetime_id,Path
Index,,,,,,,,,,,,
91,NMC6,NMC,gost45,Заявка №57 KAMAZ 006-007,1,1. Пункт 1-4.1,6,2,133,1,20240314161930,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
95,NMC7,NMC,gost45,Заявка №57 KAMAZ 006-007,1,1. Пункт 1-4.1,7,2,133,2,20240314161946,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
22,NMC10,NMC,CCroom,"Заявка № 60 Образцы C-NMC № 010, 011",1,1. Пункт 1-4.1,10,[],131,1,20240325100127,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
28,NMC11,NMC,CCroom,"Заявка № 60 Образцы C-NMC № 010, 011",1,1. Пункт 1-4.1,11,[],131,2,20240325100138,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
34,NMC12,NMC,gost60,Заявка№ 61,1,1. п 1- п4.1,12,3,131,3,20240325101703,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
...,...,...,...,...,...,...,...,...,...,...,...,...
59,NMC13,NMC,gost60,Заявка№ 61,4,4. Испытание рабочих характеристик,13,3,131,6,20240520100859,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
64,LTO3,LTO,gost60,7. Заявка № 71 К3 К13,4,4. Испытание рабочих характеристик,3,3,131,7,20240520101600,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
77,LTO13,LTO,gost60,7. Заявка № 71 К3 К13,4,4. Испытание рабочих характеристик,13,3,132,4,20240520101645,D:\!Science\Физтех\Циклирования\28.05.24 - рез...


In [5]:
def create_statistics(result_slice:pd.DataFrame, statistic_pattern):
    data = [bp.import_xls(filename, data_name_pattern='record', temp_name_pattern='auxTemp', temp_column_pattern='T1') for filename in result_slice['Path']]
    dataset = pd.concat(data,ignore_index=True)
    bp.rename_columns(dataset, {'Power(W)':'P'})
    bp.parse_time(dataset, time_column='Time', datetime_column='Datetime')
    bp.parse_time(dataset, time_column='Total Time')
    x = dataset['Total Time'].diff()
    cracks = x[x < 0]
    for idx, value in cracks.items():
        break_time = dataset['Datetime'][idx]-dataset['Datetime'][idx-1]
        dataset['Total Time'].loc[idx:] += break_time.total_seconds() + dataset['Total Time'][idx-1]


    statistics = bp.generate_statistics(dataset, group_marker=(dataset['Time'] < dataset['Time'].shift()).cumsum(), statistics_pattern=statistic_pattern)
    statistics['Net Q'] = np.sign(statistics['I_mean'])*statistics['Q_max']
    statistics['Net Energy'] = np.sign(statistics['I_mean'])*statistics['Energy_max']
    bp.rename_columns(statistics, {'Step Type_unique_values':'Step Type'})
    statistics['Step Type'] = statistics['Step Type'].str.lower()

    return statistics, dataset

def check_time_consequence(df, column='Total Time_min'):
    if np.where(statistics[column] < statistics[column].shift(), )[0].size >0:
        raise ValueError('Not consistent data!')

In [6]:
slices = [i for i in result.groupby(['ID', 'Step_id'])]
slices[63]

(('LTO7', 4),
          ID Chem Protocol                 Tech  Step_id  \
 Index                                                     
 99     LTO7  LTO   Imax45  6. Заявка №70 К6 К7        4   
 
                                      Step  Pouch Thermos  Potentiostat  \
 Index                                                                    
 99     4. Испытание рабочих характеристик      7       1           133   
 
        Channel     Datetime_id  \
 Index                            
 99           3  20240513135855   
 
                                                     Path  
 Index                                                     
 99     D:\!Science\Физтех\Циклирования\28.05.24 - рез...  )

In [13]:
statistic_pattern = {'Step Type':'unique_values',
                     'I':['mean', 'std'],
                     'Time':['range', 'diff'],
                     'Q':['max'],
                     'E':['first', 'last','mean'],
                     'T':['mean', 'min', 'max'],
                     'Datetime':['min', 'max'],
                     'Total Time':'min',
                     'Energy':'max',
                     'P':['mean', 'last']}

In [ ]:


for slice in slices:
    pouch, step = slice[0]
    print(pouch, step)
    statistics, data = create_statistics(slice[1], statistic_pattern)
    check_time_consequence(statistics,)
    save_path = fr'D:\!Science\Физтех\Циклирования\Результаты\Статистика\{step}\{pouch}.csv'
    save_data_path = fr'D:\!Science\Физтех\Циклирования\Результаты\Данные\{step}\{pouch}.csv'
    bp.exporting.save_experiment(statistics, save_path)
    bp.exporting.save_experiment(data, save_data_path)

In [11]:
statistics

,Step Type,I_mean,I_std,Time_range,Time_diff,Q_max,E_first,E_last,E_mean,T_mean,T_min,T_max,Datetime_min,Datetime_max,Total Time_min,Energy_max,P_mean,P_last,Net Q,Net Energy
0,cc dchg,-17.993014,0.002701,52.0,8.666667,0.2614,3.1399,2.4996,2.841386,28.771429,28.6,29.1,2024-05-13 13:43:50,2024-05-13 13:44:42,0.0,0.7560,-51.125429,-44.9713,-0.2614,-0.7560
1,rest,0.000000,0.000000,1800.0,10.000000,0.0000,2.5367,3.0016,2.959207,27.730387,26.7,29.2,2024-05-13 13:44:43,2024-05-13 14:14:43,52.0,0.0000,0.000000,0.0000,0.0000,0.0000
2,cccv chg,16.072389,4.972748,4097.0,9.992683,18.3225,3.0329,4.2004,3.828520,28.518491,25.8,30.2,2024-05-13 14:14:43,2024-05-13 15:23:00,1852.0,69.3524,60.818700,3.7751,18.3225,69.3524
3,cc dchg,-17.995147,0.000866,3624.0,9.983471,18.1181,4.1813,2.4989,3.667616,29.431044,28.3,33.0,2024-05-13 15:23:00,2024-05-13 16:23:24,5949.0,66.4972,-65.999242,-44.9695,-18.1181,-66.4972
4,rest,0.000000,0.000000,1800.0,10.000000,0.0000,2.5233,3.1586,3.125591,30.379006,28.5,33.4,2024-05-13 16:23:24,2024-05-13 16:53:24,9573.0,0.0000,0.000000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064,cc dchg,-1.794800,NaN,0.0,NaN,0.0774,2.4996,2.4996,2.499600,25.4,25.4,25.4,2024-05-19 04:57:51,2024-05-19 04:57:51,486845.0,0.2117,-4.486300,-4.4863,-0.0774,-0.2117
1065,rest,0.000000,0.000000,10800.0,10.000000,0.0000,2.5034,2.6955,2.671606,26.715911,25.3,27.8,2024-05-19 04:57:51,2024-05-19 07:57:51,486845.0,0.0000,0.000000,0.0000,0.0000,0.0000
1066,cc dchg,-1.794505,0.001008,43.0,1.000000,0.0214,2.6952,2.5042,2.601175,27.095455,26.9,27.1,2024-05-19 07:57:51,2024-05-19 07:58:34,497645.0,0.0557,-4.667791,-4.4947,-0.0214,-0.0557
1067,cc dchg,-1.794700,NaN,0.0,NaN,0.0219,2.4997,2.4997,2.499700,27.1,27.1,27.1,2024-05-19 07:58:35,2024-05-19 07:58:35,497689.0,0.0569,-4.486100,-4.4861,-0.0219,-0.0569


In [ ]:
slice = bp.exporting.load_experiment(save_path, parse_dates = ['Datetime_min', 'Datetime_max'])
slice['Step Type'].str.findall('.*dchg')

In [137]:
slice_1 = result[result['Step_id']==3][result['ID']=='NMC10']
slice_1

C:\Users\sk804\AppData\Local\Temp\ipykernel_12556\667288033.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  slice_1 = result[result['Step_id']==3][result['ID']=='NMC10']


,ID,Chem,Protocol,Tech,Step_id,Step,Pouch,Thermos,Potentiostat,Channel,Datetime_id,Path
Index,,,,,,,,,,,,
67,NMC10,NMC,CCroom,"Заявка № 60 Образцы C-NMC № 010, 011",3,3. Пункт 5 Циклирование 28 суток,10,[],132,1,20240404103731,"D:\!Science\Физтех\Циклирования\28.05.24 - результаты\NMC\Заявка № 60 Образцы C-NMC № 010, 011\3. Пункт 5 Циклирование 28 суток\010\до сбоя\BTS82-132-1-1-113-20240404103731-Циклирование 28 суток.xlsx"
68,NMC10,NMC,CCroom,"Заявка № 60 Образцы C-NMC № 010, 011",3,3. Пункт 5 Циклирование 28 суток,10,[],132,1,20240411142851,"D:\!Science\Физтех\Циклирования\28.05.24 - результаты\NMC\Заявка № 60 Образцы C-NMC № 010, 011\3. Пункт 5 Циклирование 28 суток\010\после сбоя\BTS82-132-1-1-116-20240411142851-п5.xlsx"


In [140]:
# Проверка что всё время в статистике идёт подряд и нет скачков.
slices =[i for i in  result.groupby(['ID', 'Step_id'])]
slice = slices[10]
# pouch, step = slice_1[0]
# print(pouch, step)
statistics, data = create_statistics(slice_1, statistic_pattern)

LFP3 3


D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\Python\Projects\Battery_Parser\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [54]:
np.where(statistics['Datetime_min'] < statistics['Datetime_min'].shift(), )[0]

statistics[75500:75520]

,Step Type,I_mean,I_std,Time_range,Time_diff,Q_max,E_first,E_last,E_mean,T_mean,T_min,T_max,Datetime_min,Datetime_max,Energy_max,P_mean,P_last,Net Q,Net Energy
75500,cp chg,21.246444,0.140481,8.0,1.0,0.0471,3.2393,3.3070,3.293433,44.866667,44.8,44.9,2024-05-09 04:53:09,2024-05-09 04:53:17,0.1555,69.971044,69.9536,0.0471,0.1555
75501,cp dchg,-21.353391,0.034357,32.0,1.0,0.1899,3.3072,3.2740,3.275994,44.827273,44.8,44.9,2024-05-09 04:53:17,2024-05-09 04:53:49,0.6218,-69.953394,-69.9593,-0.1899,-0.6218
75502,cp chg,42.137178,0.244954,8.0,1.0,0.0935,3.2740,3.3357,3.322200,44.788889,44.7,44.8,2024-05-09 04:53:49,2024-05-09 04:53:57,0.3111,139.984044,139.9583,0.0935,0.3111
75503,rest,0.000000,0.000000,44.0,1.0,0.0000,3.3101,3.3073,3.305876,44.802222,44.7,45.0,2024-05-09 04:53:57,2024-05-09 04:54:41,0.0000,0.000000,0.0000,0.0000,0.0000
75504,rest,0.000000,0.000000,16.0,1.0,0.0000,3.3073,3.3084,3.307829,44.788235,44.7,44.8,2024-05-09 04:54:41,2024-05-09 04:54:57,0.0000,0.000000,0.0000,0.0000,0.0000
75505,cp dchg,-10.601572,0.007603,28.0,1.0,0.0825,3.3084,3.2946,3.297345,44.8,44.8,44.8,2024-05-09 04:54:57,2024-05-09 04:55:25,0.2719,-34.957007,-34.9599,-0.0825,-0.2719
75506,cp dchg,-21.299454,0.020659,12.0,1.0,0.0710,3.2946,3.2815,3.284631,44.8,44.8,44.8,2024-05-09 04:55:25,2024-05-09 04:55:37,0.2332,-69.960769,-69.9666,-0.0710,-0.2332
75507,cp chg,10.577322,0.034052,8.0,1.0,0.0235,3.2814,3.3116,3.305267,44.8,44.8,44.8,2024-05-09 04:55:37,2024-05-09 04:55:45,0.0777,34.960578,34.9655,0.0235,0.0777
75508,rest,0.000000,0.000000,16.0,1.0,0.0000,3.3052,3.3062,3.305329,44.8,44.8,44.8,2024-05-09 04:55:45,2024-05-09 04:56:01,0.0000,0.000000,0.0000,0.0000,0.0000
75509,cp dchg,-10.605400,0.006998,24.0,1.0,0.0707,3.3062,3.2935,3.295900,44.796,44.7,44.8,2024-05-09 04:56:01,2024-05-09 04:56:25,0.2330,-34.954308,-34.9420,-0.0707,-0.2330


In [111]:
# dataset = pd.concat(data,ignore_index=True)
# bp.rename_columns(dataset, {'Power(W)':'P'})
# bp.parse_time(dataset, time_column='Time', datetime_column='Datetime')
# bp.parse_time(dataset, time_column='Total Time')
# statistics = bp.generate_statistics(dataset, group_marker=(dataset['Time'] < dataset['Time'].shift()).cumsum(), statistics_pattern=statistic_pattern)
# statistics['Net Q'] = np.sign(statistics['I_mean'])*statistics['Q_max']
# statistics['Net Energy'] = np.sign(statistics['I_mean'])*statistics['Energy_max']
# bp.rename_columns(statistics, {'Step Type_unique_values':'Step Type'})
# statistics['Step Type'] = statistics['Step Type'].str.lower()


In [7]:
data_path = r'D:\!Science\Физтех\Циклирования\Результаты\Данные'
slices = [i for i in result.groupby(['ID', 'Step_id'])]
data = {}
for slice in slices:
    pouch, step = slice[0]
    print(pouch, step)
    filepath = fr'{data_path}\{step}\{pouch}.csv'
    data[slice[0]] = bp.exporting.load_experiment(filepath, parse_dates = ['Datetime'])

LFP1 1
LFP1 2
LFP1 3
LFP1 4
LFP2 1
LFP2 2
LFP2 3
LFP2 4
LFP3 1
LFP3 2
LFP3 3
LFP3 4
LFP4 1
LFP4 2
LFP4 3
LFP4 4
LFP5 1
LFP5 2
LFP5 3
LFP5 4
LFP6 1
LFP6 2
LFP6 3
LFP6 4
LFP7 1
LFP7 2
LFP7 3
LFP7 4
LFP8 1
LFP8 2
LFP8 3
LFP8 4
LTO11 1
LTO11 2
LTO11 3
LTO11 4
LTO12 1
LTO12 2
LTO12 3
LTO12 4
LTO13 1
LTO13 2
LTO13 3
LTO13 4
LTO3 1
LTO3 2
LTO3 3
LTO3 4
LTO4 1
LTO4 2
LTO4 3
LTO4 4
LTO5 1
LTO5 2
LTO5 3
LTO5 4
LTO6 1
LTO6 2
LTO6 3
LTO6 4
LTO7 1
LTO7 2
LTO7 3
LTO7 4
NMC10 1
NMC10 2
NMC10 3
NMC10 4
NMC11 1
NMC11 2
NMC11 3
NMC11 4
NMC12 1
NMC12 2
NMC12 3
NMC12 4
NMC13 1
NMC13 2
NMC13 3
NMC13 4
NMC14 1
NMC14 2
NMC14 3
NMC14 4
NMC15 1
NMC15 2
NMC15 3
NMC15 4
NMC6 1
NMC6 2
NMC6 3
NMC6 4
NMC7 1
NMC7 2
NMC7 3
NMC7 4


In [24]:
for key, dataset in data.items():
    pouch, step = key
    statistics = bp.generate_statistics(dataset, group_marker=[(dataset['Time'] < dataset['Time'].shift()).cumsum(), (dataset['Step Type'] != dataset['Step Type'].shift()).cumsum()], statistics_pattern=statistic_pattern)
    statistics['Net Q'] = np.sign(statistics['I_mean'])*statistics['Q_max']
    statistics['Net Energy'] = np.sign(statistics['I_mean'])*statistics['Energy_max']
    bp.rename_columns(statistics, {'Step Type_unique_values':'Step Type'})
    statistics['Step Type'] = statistics['Step Type'].str.lower()
    statistics.reset_index(drop=True, inplace=True)
    check_time_consequence(statistics,)
    save_path = fr'D:\!Science\Физтех\Циклирования\Результаты\Статистика\{step}\{pouch}.csv'
    bp.exporting.save_experiment(statistics, save_path)